In [ ]:
# UTILSSSS
%load_ext autoreload
%autoreload 2
import torch
import os, time 
import cv2
from torch.autograd import Variable

import solver_expnet as se
import model_expnet as me
import loadOpenFace as ff
import numpy as np

def ReadImage(pathname, isFacenetData = False):
        img = cv2.imread(pathname)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.resize(img, (96, 96), interpolation=cv2.INTER_LINEAR)
        img = np.transpose(img, (2, 0, 1))
        img = img.astype(np.float32) / 255.0
        I_ = torch.from_numpy(img).unsqueeze(0)
        if isFacenetData:
            
            if useCuda:
                I_ = I_.cuda()
        return I_

def getImage(folder):
        img_paths = [0]
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    img_paths.insert(-1, file_path)
                elif os.path.isdir(file_path):
                    getImage(file_path)
            except Exception as e:
                print(e)
        return img_paths[0:-1]


def prepareOpenFace(useCuda=True, gpuDevice=0, useMultiGPU=False):
    print("Preparing openface object")
    model = ff.netOpenFace(useCuda, gpuDevice)
    model.load_state_dict(torch.load(os.path.join('./', 'openface.pth')))

    if useMultiGPU:
        model = nn.DataParallel(model)

    return model


In [ ]:
# Running openface network.

useCuda = True
if useCuda:
    assert torch.cuda.is_available()
else:
    assert False, 'Sorry, .pth file contains CUDA version of the network only.'

nof = prepareOpenFace()
nof = nof.eval()

I = np.reshape(np.array(range(96 * 96), dtype=np.float32) * 0.01, (1,96,96))
I = np.concatenate([I, I, I], axis=0)
I_ = torch.from_numpy(I).unsqueeze(0)

if useCuda:
    I_ = I_.cuda()

I_ = Variable(I_)

img_paths = getImage("data/")
imgs = []
for img_path in img_paths[0:2]:
    imgs.append(ReadImage(img_path, True))
print()   
I_ = torch.cat(imgs, 0)
I_ = Variable(I_, requires_grad=True)

start = time.time()
f_736 = nof(I_)

targets = []
for w in f_736:
    w_ = w.view(-1, 1)
    x = torch.ones(w_.size(0))
    w_t = torch.transpose(w_, 1,0)
    target = np.dot(w_t.data, x)
    
    targets.append(torch.from_numpy(target).type(torch.FloatTensor).cuda())

#print(targets)

# Gib=ving me output in the shape of <number of images>*736*3*3 i.e 7*736*3*3 (dev case)
print(f_736[0]) 

print("  + Forward pass took {} seconds.".format(time.time() - start))


In [ ]:

img_paths = getImage("data/")
train_data = []
for i, img_path in enumerate(img_paths[0:2]):
    train_data.append([ReadImage(img_path, False).type(torch.FloatTensor).cuda(), f_736[i]])

print(train_data)
    
def loss_expnet(input_, target_):
    #print(input_)
    #print(target_)
    ls = (target_-input_).view(-1)
    ls = torch.sum(ls**2)
    #loss = np.log(ls)
    #print(loss)
    return ls



In [ ]:
import solver_expnet as se
import model_expnet as me
#187, 164
#167, 147
#151, 134
#55, 48 after 15000 epoch

model = me.ExpNet(useCuda=True, gpuDevice=0)
solver = se.Solver(optim_args={"lr": 1e-7}, loss_func=loss_expnet)

solver.train(model, train_data, train_data, log_nth=1, num_epochs=15000)